In [1]:
%%vertica
SELECT 'Connected to Vertica!'

,?column?
0,Connected to Vertica!


In [2]:
%vertica SET ROLE restricted

,?column?


In [3]:
%%vertica
SET TIME ZONE TO UTC

,?column?


In [5]:
%%vertica -> week_emails
SELECT
    FLOOR(DATEDIFF('day', content_availability, CURRENT_DATE()) / 7) :: INT AS week_number,
    segments.course_id AS course_id,
    user_username AS student_name,
    user_email,
    content_availability,
    c.course_name AS course_name
FROM
    ret.exp6_enrollments AS segments
LEFT JOIN
    lms_read_replica.bulk_email_optout AS eo
ON
    eo.user_id = segments.user_id
LEFT JOIN
    production.d_user_course duc
ON
    duc.user_id = segments.user_id AND duc.course_id = segments.course_id
JOIN
    production.d_user AS du
ON
    du.user_id = segments.user_id
JOIN
    (SELECT DISTINCT course_id, course_name, course_end_date FROM business_intelligence.course_master) AS c
ON
    c.course_id = segments.course_id
WHERE
        segment_id = 1
    AND eo.user_id IS NULL
    AND duc.current_enrollment_is_active
    AND c.course_end_date > NOW()
ORDER BY
    1, 2, 3

This next cell should just return a single week number with all of the users in it.

In [6]:
week_emails.groupby('week_number').user_email.count()

week_number
3    31695
Name: user_email, dtype: int64

In [7]:
c = week_emails.groupby(['user_email', 'course_id']).user_email.count()
assert len(c[c > 1]) == 0, 'No user should be getting two emails for the same course!'

In [8]:
import edx_secret
sailthru_api_key = edx_secret.read_secret_from_env_or_prompt('OPS_SAILTHRU_API_KEY')
sailthru_api_secret = edx_secret.read_secret_from_env_or_prompt('OPS_SAILTHRU_API_SECRET')

In [9]:
TEMPLATE_NAME = 'RET Recurring Nudge'
MAX_BATCH_SIZE = 100

In [25]:
from collections import namedtuple
import textwrap
from urllib.parse import quote_plus

Template = namedtuple('Template', ['subject', 'body', 'preview', 'call_to_action', 'call_to_action_link'])

TEMPLATES = [
    Template( # Email Doc N - Reminder B
        '{course_name} has started on edX.org!',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Remember when you enrolled in <strong>{course_name}</strong> on edX.org? We do! We are excited to let you know the course has started. Come see what everyone is learning. 
        """),
        'Remember when you enrolled in <strong>{course_name}</strong> on edX.org?',
        'Start learning now',
        'https://courses.edx.org/courses/{course_id}/course/',
    ),
#     Template( # Email Doc N - Reminder C
#         'Keep going with {course_name}',
#         textwrap.dedent("""\
#             Dear {student_name},
#             <br><br>
#             You enrolled in <strong>{course_name}</strong> and it is available for you now. This is a great opportunity to keep your momentum going and continue learning.
#         """),
#         'You enrolled in <strong>{course_name}</strong> and it is available for you now.',
#         'Continue learning',
#         'https://courses.edx.org/courses/{course_id}/course/',
#     ),
    Template( # Email Doc N - Nudge (problems)
        'Solve a few problems in {course_name}',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Numerous edX learners in <strong>{course_name}</strong> are completing more problems every week. What do you want to do to keep learning? 
        """),
        'Numerous edX learners in <strong>{course_name}</strong> are completing more problems every week.',
        'See what’s new',
        'https://courses.edx.org/courses/{course_id}/course/',
    ),
    Template( # Email Doc N - Nudge mobile 2
        'Learn on the go with the edX.org App',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Many edX.org learners use their commutes to engage with courses. Want to learn on the go? Learn on your preferred device. Remember, when you participate in <strong>{course_name}</strong>, you learn more things.
        """),
        'Many edX.org learners use their commutes to engage with courses. Want to learn on the go?',
        'Get the app',
        'https://www.edx.org/mobile',
    ),
    Template( # Email Doc N - Nudge (videos)
        'Watch and learn with {course_name}',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Many successful edX learners in <strong>{course_name}</strong> are watching videos every week. What do you want to do to keep learning? 
        """),
        'Many successful edX learners in <strong>{course_name}</strong> are watching videos every week.',
        'See my course',
        'https://courses.edx.org/courses/{course_id}/course/',
    ),
    Template( # Email Doc N - Nudge (mobile)
        'Learn on the go with the edX.org App',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Looking for time to learn on the go? Try the <a href="https://www.edx.org/mobile">edX.org App</a>. Spending more time in <strong>{course_name}</strong> means you have a better chance of learning something that makes a difference in your life.
        """),
        'Looking for time to learn on the go? Try the <a href="https://www.edx.org/mobile">edX.org App</a>.',
        'Get the app',
        'https://www.edx.org/mobile',
    ),
    Template( # Email Doc N - Nudge (discussions)
        'Talk up {course_name} at edX.org',
        textwrap.dedent("""\
            Dear {student_name},
            <br><br>
            Lots of edX learners in <strong>{course_name}</strong> are talking and asking questions in the discussion forums. What do you want to do to keep learning? 
        """),
        'Lots of edX learners in <strong>{course_name}</strong> are talking and asking questions in the discussion forums.',
        'See my course',
        'https://courses.edx.org/courses/{course_id}/course/',
    ),
#     Template( # Email Doc N - Nudge (finding time)
#         'Find balance and learn more in  {course_name}',
#         textwrap.dedent("""\
#             Dear {student_name},
#             <br><br>
#             Many edX learner learners are juggling jobs, family obligations, and other responsibilities while learning in <strong>{course_name}</strong>. Setting aside a few minutes for your learning at the same time each day can help you balance all the things you do.
#         """),
#         'Many edX learner learners are juggling jobs, family obligations, and other responsibilities while learning in <strong>{course_name}</strong>.',
#         'Keep learning',
#         'https://courses.edx.org/courses/{course_id}/course/',
#     ),
]

def as_template_vars(week, row):
    template = TEMPLATES[week]
    row_vals = row._asdict()
    
    assert row_vals.get('student_name'), f'student_name is empty for {row_vals}'
    assert row_vals.get('course_name'), f'course_name is empty for {row_vals}'
    assert row_vals.get('course_id'), f'course_id is empty for {row_vals}'

    row_vals['course_id'] = quote_plus(row_vals['course_id'])
    
    result = {
        'ret_pacing_body': template.body.format(**row_vals),
        'ret_pacing_course_id': row.course_id,
        'ret_pacing_subject_line': template.subject.format(**row_vals),
        'ret_pacing_preview_text': template.preview.format(**row_vals),
        'ret_pacing_cta_text': template.call_to_action.format(**row_vals),
        'ret_pacing_cta_link': template.call_to_action_link.format(**row_vals),
    }
    if week in (2, 4):
        result['ret_pacing_secondary_body'] = 'Not ready for the app now?'
        result['ret_pacing_secondary_cta_text'] = 'See my course'
        result['ret_pacing_secondary_cta_link'] = 'https://courses.edx.org/courses/{course_id}/course/'.format(**row_vals)
    
    return result

# Send email kill switch

When this is set to `False` no emails will be sent.

In [26]:
SEND_EMAIL = True

In [27]:
from sailthru.sailthru_client import SailthruClient

sc = sailthru_client = SailthruClient(sailthru_api_key, sailthru_api_secret)

Vars in the template - the email will not be sent (fail silently) if these are not set!

* ret_pacing_body
* ret_pacing_course_id
* ret_pacing_subject_line
* ret_pacing_preview_text
* ret_pacing_cta_text

In [28]:
import itertools
import time
from pprint import pprint as pp
import numpy as np
from functools import partial
import json
from ipywidgets import IntProgress
from IPython.display import display

class SailthruEmail(object):
    
    def __init__(self, sailthru_client, template_vars=None):
        self.sailthru_client = SailthruClient(sailthru_api_key, sailthru_api_secret)
        self.template_vars = template_vars or {}
        
    def send_to_users(self, var_overrides, batch_size=MAX_BATCH_SIZE, is_test=False):
        # This is the max batch size that Sailthru supports
        assert(batch_size <= MAX_BATCH_SIZE)
        
        progress_bar = IntProgress(min=0, max=len(var_overrides))
        display(progress_bar)
        
        sent_count = 0
        by_week = var_overrides.set_index('week_number')
        for week in by_week.index.unique():
        
            df = by_week.loc[week]
            
            for k, batch in df.groupby(np.arange(len(df))//batch_size):
                sent_count += self._send_batch(batch, is_test=is_test, progress_bar=progress_bar)
        
        return sent_count
            
    def _send_batch(self, batch, is_test=False, progress_bar=None):
        self.wait_for_rate_limit()
        
        week = batch.index.unique()[0]
        # Build the template vars for every user in the batch
        batch_vars = {
            row.user_email: {
                **self.template_vars,
                **as_template_vars(week, row)
            }
            for row in batch.itertuples()
        }

        for email, override in batch_vars.items():
            assert(email)
            assert(override['ret_pacing_body'])
            assert(override['ret_pacing_course_id'])
            assert(override['ret_pacing_subject_line'])
            assert(override['ret_pacing_preview_text'])
            assert(override['ret_pacing_cta_text'])
        
        sent_count = 0
        if SEND_EMAIL:
            options = {}
            if is_test:
                options['test'] = 1
                
            emails = [row.user_email for row in batch.itertuples()]
            
            response = sc.multi_send(
                TEMPLATE_NAME,
                emails,
                evars=batch_vars,
                options=options
            )
            if response.is_ok():
                body = response.get_body()
                sent_count = int(body.get("sent_count", 1))
                print(f'Send successful to {sent_count} users')
            else:
                error = response.get_error()
                print("Error: " + error.get_message())
                print("Status Code: " + str(response.get_status_code()))
                print("Error Code: " + str(error.get_error_code()))
                print("Failed batch: {0}".format(json.dumps(emails)))
        else:
            batch_size = len(batch_vars)
            sent_count = batch_size
            print(
                f'Would have sent an email{" [TEST]" if is_test else ""}'
                f' using the template "{TEMPLATE_NAME}" for week {week} to {batch_size} users'
            )
            print(json.dumps(batch_vars, indent=4))
        
        if progress_bar is not None:
            progress_bar.value += len(batch_vars)
        
        return sent_count

    def wait_for_rate_limit(self):
        rate_limit_info = self.sailthru_client.get_last_rate_limit_info('send', 'POST')
        if rate_limit_info is not None:
            limit = int(rate_limit_info['limit'])
            remaining = int(rate_limit_info['remaining'])
            reset_timestamp = int(rate_limit_info['reset'])
            seconds_till_reset = reset_timestamp - time.time()
            if remaining <= 0 and seconds_till_reset > 0:
                print(f'Rate limit exceeded, sleeping for {seconds_till_reset} seconds')
                time.sleep(seconds_till_reset + 1)

In [29]:
email = SailthruEmail(
    sailthru_client=sailthru_client,
    template_vars={
    }
)

# Sending emails

1. First send to RET
2. Send to edX employees
3. Set SEND_EMAIL=False
4. Call the method to send to all learners - this will validate all of template vars
5. Set SEND_EMAIL=True
6. Send the email to all learers

In [15]:
%%vertica -> edx_usernames
SELECT
    user_email,
    user_username
FROM production.d_user
WHERE
    user_email LIKE '%@edx.org'

In [16]:
test_users = edx_usernames.set_index('user_email')

In [30]:
import pandas as pd

def send_email_to_ret(week_number):
    test_emails = [
        'gabe@edx.org',
        'cblackburn@edx.org',
        'cale@edx.org',
        'nasthagiri@edx.org',
        'aboehm@edx.org',
        'ddomingos@edx.org',
        'kwillemin@edx.org',
        'jzheng@edx.org'
    ]
    usernames = [test_users.loc[e].user_username for e in test_emails]

    test_emails_df = pd.DataFrame({
        'week_number': [week_number] * len(test_emails),
        'course_id': ['course-v1:UBCx+HtC2x+2T2017'] * len(test_emails),
        'student_name': usernames,
        'user_email': test_emails,
        'course_name': ['HtC2x How to Code: Complex Data'] * len(test_emails),
    })

    return email.send_to_users(test_emails_df, is_test=True)

result = send_email_to_ret(week_number=3)
print(f'Sent to {result} users')

Send successful to 8 users
Sent to 8 users


In [ ]:
edx_emails = week_emails[week_emails.user_email.apply(lambda x: '@edx.org' in x)]
#email.send_to_users(edx_emails, is_test=True)

# Don't move your laptop once you run this next cell!

In [31]:
email.send_to_users(week_emails)

Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 98 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 1

Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 99 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 100 users
Send successful to 95 users


31646

Save a CSV with the data used to send the emails... we might want this later? This would be our only record of what emails were sent to which users.

In [32]:
#week_emails.to_csv('week_3_sent_emails.csv')

Insert into the email_experiment table the set of users included in this experiment. Note that this assumes a single cohort. If we run a second cohort, then we would need to rejigger things here a bit.

In [144]:
# %%vertica

# INSERT INTO ret.email_experiment
# SELECT
#     6 AS experiment_id,
#     'Recurring generic nudges' AS experiment_name,
#     du.user_email AS email,
#     du.user_id,
#     du.user_username AS student_name,
#     segments.course_id AS course_id,
#     c.course_name AS course_title,
#     segment_id AS group_id,
#     CASE segment_id WHEN 0 THEN 'Control'
#                     WHEN 1 THEN 'Treatment'
#     END AS group_name
# FROM
#     ret.exp6_enrollments AS segments
# LEFT JOIN
#     lms_read_replica.bulk_email_optout AS eo
# ON eo.user_id = segments.user_id
# JOIN
#     production.d_user AS du
# ON du.user_id = segments.user_id
# JOIN business_intelligence.course_master AS c
# ON c.course_id = segments.course_id
# WHERE
#     eo.user_id IS NULL

In [145]:
#%vertica COMMIT